In [5]:
import pickle
import pandas as pd

from sklearn import preprocessing
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import LSTM

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [6]:
with open('potential_customer.pickle', 'rb') as f:
    potenotal_customer_DB = pickle.load(f)

In [9]:
potenotal_customer_DB.keys()

dict_keys(['홍대', '강남', '신사', '천호', '종가', '서울대입구', '노원', '성수'])